# 01a: Practice — Build a Feed‑Forward Network (FFN) from Scratch

This notebook is a hands‑on scaffold to implement a fully‑connected feed‑forward neural network step by step using NumPy. Each section provides:

- Empty functions to fill in (raise `NotImplementedError`)
- Clear type hints and expected shapes/dtypes
- Concise algorithm guidance + math in LaTeX
- Historical notes about when/why ideas emerged

By the end, you’ll be able to train a small FFN on MNIST using the utilities in this repo.

## Historical Context

- Perceptron (single linear threshold unit): Frank Rosenblatt, 1957. Momentum grew with hardware of the time but limited by linear separability.
- Perceptron limits: Marvin Minsky & Seymour Papert, 1969, highlighted non‑linearly separable problems (e.g., XOR).
- Backpropagation (general idea/chain rule): Paul Werbos (1974, thesis); popularized by Rumelhart, Hinton, Williams (1986). Compute improvements and matrix libraries made multilayer training practical.
- Softmax + cross‑entropy (multiclass logistic regression) widely adopted by late 1980s/1990s; normalized exponential/"softmax" connections in statistical modeling matured in this era.
- Universal Approximation: Cybenko (1989); Hornik (1989–1991).
- ReLU nonlinearity: used earlier in neuroscience; popularized for deep nets by Nair & Hinton (2010) and Glorot et al. (2011), enabling better gradient flow.
- Initialization: Glorot/Xavier (2010) and He (2015) stabilizing signal propagation at scale.
- Optimization: SGD formalized by Robbins & Monro (1951); momentum by Polyak (1964); Adam by Kingma & Ba (2014).
- Modern deep learning acceleration: commodity GPUs (mid‑late 2000s), AlexNet (2012) demonstrated compute + data + architecture synergy.

## Notation and Math

We consider an L‑layer MLP (L hidden layers; output layer counts as layer L+1):

- Input: $X \in \mathbb{R}^{N \times d_0}$ (mini‑batch of N)
- Weights: $W^{(l)} \in \mathbb{R}^{d_{l-1} \times d_l}$, biases: $b^{(l)} \in \mathbb{R}^{d_l}$
- Pre‑activation: $Z^{(l)} = X^{(l-1)} W^{(l)} + b^{(l)}$
- Activation: $A^{(l)} = f(Z^{(l)})$ (ReLU in hidden layers)
- Output logits: $O = A^{(L)} W^{(L+1)} + b^{(L+1)}$
- Softmax: $P_i = \mathrm{softmax}(O_i) = \frac{e^{O_i - \mathrm{max}(O_i)}}{\sum_j e^{O_{ij} - \mathrm{max}(O_i)}}$
- Cross‑entropy: $\mathcal{L} = - \frac{1}{N} \sum_i \log P_{i,y_i}$

Key gradients for training with backpropagation:

- For softmax + cross‑entropy with integer labels $y$: $\frac{\partial \mathcal{L}}{\partial O} = \frac{1}{N}(P - \mathrm{one\text{-}hot}(y))$
- Linear layer: given $Y = XW + b$ and upstream $G = \partial \mathcal{L} / \partial Y$, then\
$\quad \partial \mathcal{L}/\partial W = X^\top G, \quad \partial \mathcal{L}/\partial b = \sum_{i=1}^N G_i, \quad \partial \mathcal{L}/\partial X = G W^\top$
- ReLU: $\mathrm{ReLU}(z)=\max(0,z)$, grad mask $M=\mathbf{1}[z>0]$, so $\partial \mathcal{L}/\partial Z = (\partial \mathcal{L}/\partial A) \odot M$

In [ ]:
# Setup
from __future__ import annotations
import math
from typing import Dict, List, Tuple

import numpy as np
from numpy.typing import NDArray

np.random.seed(42)
Float = np.float32
Int = np.int64

# Optional: dataset utilities from this repo (MNIST)
from mlp import data_providers


## 1) Initialization

Goal: create parameter dictionaries with correct shapes/dtypes and variance‑scaled random init.

- Use `float32` everywhere for speed and to mirror practical training.
- For each layer l: `W_l: (d_{l-1}, d_l)`, `b_l: (d_l,)`.
- Xavier/Glorot (2010) normal: $\sigma = \sqrt{2/(fan\_in + fan\_out)}$
- He (2015) normal: $\sigma = \sqrt{2/fan\_in}$ (good for ReLU).

Historical note: before variance‑scaled inits, deeper nets suffered from vanishing/exploding signals; Glorot (2010) and He (2015) made stable deep training much more accessible.

In [ ]:
def initialize_parameters(layer_sizes: List[int], method: str = 'he', seed: int | None = 42) -> Dict[str, NDArray[Float]]:
    """
    Create parameters for an MLP.

    Args:
        layer_sizes: [d_0, d_1, ..., d_L, d_{L+1}] including input and output dims.
        method: 'xavier' or 'he'.
        seed: optional RNG seed for reproducibility.

    Returns: dict with keys 'W1','b1',...,'W{L+1}','b{L+1}', each NDArray[float32].

    Algorithm (fill in):
        - For each l in 1..L+1, set fan_in=layer_sizes[l-1], fan_out=layer_sizes[l].
        - If method == 'xavier': std = sqrt(2.0 / (fan_in + fan_out)).
          If method == 'he':     std = sqrt(2.0 / fan_in).
        - W_l ~ Normal(0, std^2), dtype float32.
        - b_l = zeros(fan_out, dtype float32).
    """
    raise NotImplementedError


## 2) Linear Layer (Affine Transform)

$Z = XW + b$

- Shapes: `X: (N, d_in)`, `W: (d_in, d_out)`, `b: (d_out,)` -> `Z: (N, d_out)`
- Dtypes: `float32`
- Implement vectorized matrix multiply and bias add.

Historically, fast BLAS libraries and GPUs massively accelerated this step, enabling practical deep nets.

In [ ]:
def linear_forward(X: NDArray[Float], W: NDArray[Float], b: NDArray[Float]) -> NDArray[Float]:
    """Compute Z = X @ W + b, returning float32 with shape (N, d_out).
    Args: X(N,d_in), W(d_in,d_out), b(d_out,) all float32.
    """
    raise NotImplementedError


## 3) Nonlinearity (ReLU)

$\mathrm{ReLU}(z) = \max(0, z)$

- Forward: elementwise max.
- Backward: mask where $z>0$. Popularized ~2010–2011, ReLU improved gradient flow vs. sigmoids/tanh in deep nets.

In [ ]:
def relu_forward(Z: NDArray[Float]) -> NDArray[Float]:
    """Return A = max(Z, 0) with same shape/dtype.
    """
    raise NotImplementedError

def relu_backward(dA: NDArray[Float], Z: NDArray[Float]) -> NDArray[Float]:
    """Given upstream dA and preactivation Z, return dZ = dA * (Z>0).
    Shapes: both (N, d), dtype float32.
    """
    raise NotImplementedError


## 4) Softmax and Cross‑Entropy

Use numerically stable softmax by subtracting per‑row max.

$P_i = \mathrm{softmax}(O_i) = \frac{e^{O_i - m_i}}{\sum_j e^{O_{ij} - m_i}}, \quad m_i = \max_j O_{ij}$

Cross‑entropy with integer labels $y$: $\mathcal{L} = -\tfrac{1}{N} \sum_i \log P_{i,y_i}$.

Gradient: $\partial \mathcal{L} / \partial O = (P - one\text{-}hot(y)) / N$.

Softmax regression for multiclass classification became standard by the late 1980s/1990s.

In [ ]:
def softmax(logits: NDArray[Float]) -> NDArray[Float]:
    """Return row‑wise softmax probabilities with float32 shape (N, C).
    Use stability trick: subtract row max before exp.
    """
    raise NotImplementedError

def cross_entropy_loss(probs: NDArray[Float], y: NDArray[Int]) -> Tuple[Float, NDArray[Float]]:
    """Compute mean cross‑entropy and per‑example negative log‑likelihoods.
    Args: probs (N,C) float32 with rows summing to 1; y (N,) int64 class indices.
    Returns: (mean_loss: float32 scalar, nll: (N,) float32).
    """
    raise NotImplementedError

def softmax_ce_backward(logits: NDArray[Float], y: NDArray[Int]) -> NDArray[Float]:
    """Return dL/dlogits for softmax + mean cross‑entropy.
    Implement: probs = softmax(logits); grad = (probs - one_hot(y, C)) / N.
    Shape: (N, C), dtype float32.
    """
    raise NotImplementedError

def one_hot(y: NDArray[Int], num_classes: int) -> NDArray[Float]:
    """Return float32 one‑hot matrix of shape (N, num_classes).
    """
    raise NotImplementedError


## 5) Linear Backward

For $Y = XW + b$ with upstream $G = \partial \mathcal{L}/\partial Y$:

$\quad \partial \mathcal{L}/\partial W = X^\top G, \ \partial \mathcal{L}/\partial b = \sum_{i=1}^N G_i, \ \partial \mathcal{L}/\partial X = G W^\top$

Implement vectorized gradients with float32 types.

In [ ]:
def linear_backward(dY: NDArray[Float], X: NDArray[Float], W: NDArray[Float]) -> Tuple[NDArray[Float], NDArray[Float], NDArray[Float]]:
    """Given upstream dY and cached X, W, compute (dX, dW, db).
    Shapes: dY(N,d_out), X(N,d_in), W(d_in,d_out).
    Return: dX(N,d_in), dW(d_in,d_out), db(d_out,). All float32.
    """
    raise NotImplementedError


## 6) Model Forward/Backward (2‑Hidden Example)

Wire linear + ReLU blocks for a small MLP. Example architecture: `d0 -> d1 -> d2 -> C`.
- Forward should return logits and caches needed for backward.
- Backward should produce grads dict matching params keys.

Backpropagation was computationally onerous in early decades; maturing compute and frameworks in the 2000s–2010s made this routine development.

In [ ]:
def ffn_forward_2layer(params: Dict[str, NDArray[Float]], X: NDArray[Float]) -> Tuple[NDArray[Float], dict]:
    """Forward pass for 2‑hidden‑layer MLP using ReLU in hidden layers.
    Expected params keys: W1,b1,W2,b2,W3,b3 for dims d0->d1->d2->C.
    Returns: (logits: (N,C), cache: dict of intermediates for backward).
    """
    raise NotImplementedError

def ffn_backward_2layer(cache: dict, y: NDArray[Int], params: Dict[str, NDArray[Float]]) -> Dict[str, NDArray[Float]]:
    """Backward pass producing grads dict with keys dW1,db1,dW2,db2,dW3,db3.
    Use softmax_ce_backward for final layer gradient.
    """
    raise NotImplementedError


## 7) Metrics and Updates

- Accuracy: `argmax` of probabilities vs labels.
- SGD update: $\theta \leftarrow \theta - \eta \cdot g$ (momentum optional — Polyak, 1964).

In [ ]:
def accuracy(probs: NDArray[Float], y: NDArray[Int]) -> Float:
    """Compute float32 accuracy in [0,1].
    """
    raise NotImplementedError

def sgd_update(params: Dict[str, NDArray[Float]], grads: Dict[str, NDArray[Float]], lr: float = 1e-2) -> None:
    """In‑place SGD update: params[key] -= lr * grads['d'+key].
    All arrays are float32; cast lr to float32 during multiplication.
    """
    raise NotImplementedError


## 8) Training Loop (MNIST)

Use the provided `mlp.data_providers.MNISTDataProvider` to fetch mini‑batches. Implement the loop once your functions work.

Pseudocode:

1. Init params with sizes `[784, 128, 64, 10]`.
2. For each epoch and batch:
   - Forward -> logits -> probs
   - Loss, accuracy
   - Backward -> grads
   - Update params (SGD)
3. Track train/valid metrics.

Compute note: large batch sizes grew practical with GPUs; historically, small batches/stochastic updates were a pragmatic default on CPUs.

In [ ]:
def train_mnist_2layer(hidden_sizes: Tuple[int,int] = (128,64), lr: float = 1e-1, epochs: int = 3, batch_size: int = 128,
                        max_train_batches: int | None = 200, max_valid_batches: int | None = 50, init: str = 'he') -> Dict[str, float]:
    """Skeleton training loop for MNIST. Fill in once primitives are implemented.
    Returns history dict with last seen metrics (you may extend).

    Steps to implement:
      1) Build data providers: train/valid via data_providers.MNISTDataProvider.
      2) Initialize params for sizes [784, h1, h2, 10].
      3) For each epoch and batch: forward, loss, backward, SGD update.
      4) Periodically evaluate on valid set.
    """
    raise NotImplementedError


## 9) Quick Sanity Checks (Suggested)

- Shapes: verify each function returns expected shapes.
- Dtypes: ensure float32 throughout to avoid accidental float64 upcasts.
- Numerical stability: confirm softmax rows sum to ~1 and no NaNs.
- Overfit a tiny subset (e.g., 256 samples) to confirm learning dynamics.

## References (Pointer‑Style)

- Rosenblatt, F. (1957). The Perceptron.
- Minsky, M., Papert, S. (1969). Perceptrons.
- Werbos, P. (1974). Beyond Regression: New Tools for Prediction and Analysis.
- Rumelhart, D., Hinton, G., Williams, R. (1986). Learning representations by back‑propagating errors.
- Cybenko, G. (1989); Hornik, K. (1989–1991): Universal approximation results.
- Glorot, X., Bengio, Y. (2010). Understanding the difficulty of training deep feedforward neural networks.
- Nair, V., Hinton, G. (2010); Glorot et al. (2011): ReLU in deep nets.
- He, K. et al. (2015). Delving deep into rectifiers.
- Kingma, D., Ba, J. (2014). Adam.
- Krizhevsky, A., Sutskever, I., Hinton, G. (2012). ImageNet classification with deep CNNs.